# Project 2 - Lab 5 - Filter and aggregate the water quality data

Recall that one of the files (starts with `mces`) contains water quality measurements for lakes in the Twin Cities.  In this lab, we will narrow down the list of lakes for which we have at least one of each measurement type (phosphorus and secchi depth) for each year between 2004 and 2015.

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import when, col
from more_pyspark import to_pandas, recode
spark = (SparkSession.builder.appName('Ops')
         .getOrCreate())
from composable.glob import glob
from composable.strict import map, star_map, filter, sorted
from composable.sequence import reduce
from composable import pipeable
from pyspark.sql.functions import lit
import pandas as pd
from composable.tuple import split_by

your 131072x1 screen size is bogus. expect trouble


22/12/06 17:34:38 WARN Utils: Your hostname, lu4543hm221 resolves to a loopback address: 127.0.1.1; using 172.21.183.83 instead (on interface eth0)
22/12/06 17:34:38 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/06 17:34:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/12/06 17:34:41 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
mces = spark.read.csv('./data/MinneMUDAC_raw_files/MinneMUDAC_raw_files/mces_lakes_1999_2014.txt', header=True, sep='\t')

## Problem 1 - Inspect the data

We will be focusing on two of water quality measurements: phosphorus and secchi depth.  Before we start trimming the data set, we should explore these metrics.

1. Each of the measures has a `QUALIFIER` column.  Group and aggregate by each of these columns and note any problematic values.  **Hint.** This search should indicate that some of the phosphorus measurements should be dropped.  Make sure you include this in your primary query.
2. Each measure also includes a `Units` column.  Check that all measurement are in the same units, and convert as needed.

In [3]:
mces.toPandas()

22/12/06 17:34:48 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


,PROJECT_ID,DATA_SET_TITLE,LAKE_NAME,CITY,COUNTY,DNR_ID_Site_Number,MAJOR_WATERSHED,WATER_PLANNING_AUTHORITY,LAKE_SITE_NUMBER,START_DATE,...,Secchi_Depth_RESULT_SIGN,Secchi_Depth_RESULT,Secchi_Depth_QUALIFIER,Secchi_Depth_Units,Total_Phosphorus_RESULT_SIGN,Total_Phosphorus_RESULT,Total_Phosphorus_QUALIFIER,Total_Phosphorus_Units,longitude,latitude
0,7108,Citizen Assisted Monitoring Program (CAMP) for...,Acorn Lake,Oakdale,Washington,82010200-01,Lower St. Croix River,Valley Branch WD,1,2006-04-16,...,None,1,Approved,m,None,0.156,Approved,mg/L,-92.97171054,45.01655642
1,7108,Citizen Assisted Monitoring Program (CAMP) for...,Acorn Lake,Oakdale,Washington,82010200-01,Lower St. Croix River,Valley Branch WD,1,2006-05-01,...,None,None,None,m,None,None,None,mg/L,-92.97171054,45.01655642
2,7108,Citizen Assisted Monitoring Program (CAMP) for...,Acorn Lake,Oakdale,Washington,82010200-01,Lower St. Croix River,Valley Branch WD,1,2006-05-02,...,None,0.66,Approved,m,None,0.107,Approved,mg/L,-92.97171054,45.01655642
3,7108,Citizen Assisted Monitoring Program (CAMP) for...,Acorn Lake,Oakdale,Washington,82010200-01,Lower St. Croix River,Valley Branch WD,1,2006-05-16,...,None,0.66,Approved,m,None,0.141,Approved,mg/L,-92.97171054,45.01655642
4,7108,Citizen Assisted Monitoring Program (CAMP) for...,Acorn Lake,Oakdale,Washington,82010200-01,Lower St. Croix River,Valley Branch WD,1,2006-05-30,...,None,0.5,Approved,m,None,0.029,Approved,mg/L,-92.97171054,45.01655642
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48252,7105,Lakes Monitoring,Zumbra Lake,Victoria,Carver,10004100-01,Mississippi River - Twin Cities,Minnehaha Creek WD,1,2002-09-16,...,None,None,None,m,None,0.224,Approved,mg/L,-93.66768906,44.88381717
48253,7105,Lakes Monitoring,Zumbra Lake,Victoria,Carver,10004100-01,Mississippi River - Twin Cities,Minnehaha Creek WD,1,2002-10-01,...,None,2.7,Approved,m,~,0.026,Approved,mg/L,-93.66768906,44.88381717
48254,7105,Lakes Monitoring,Zumbra Lake,Victoria,Carver,10004100-01,Mississippi River - Twin Cities,Minnehaha Creek WD,1,2002-10-01,...,None,None,None,m,~,0.015,Approved,mg/L,-93.66768906,44.88381717
48255,7105,Lakes Monitoring,Zumbra Lake,Victoria,Carver,10004100-01,Mississippi River - Twin Cities,Minnehaha Creek WD,1,2002-10-14,...,None,3,Approved,m,~,0.011,Approved,mg/L,-93.66768906,44.88381717


In [4]:
qualifiers1 = (mces.groupBy('Total_Phosphorus_QUALIFIER').count())
qualifiers1.toPandas()

,Total_Phosphorus_QUALIFIER,count
0,Approved,43639
1,None,4583
2,Suspect,35


In [5]:
qualifiers2 = (mces.groupBy('Secchi_Depth_QUALIFIER').count())
qualifiers2.toPandas()

,Secchi_Depth_QUALIFIER,count
0,Approved,35104
1,None,13153


In [6]:
unit = (mces.groupBy('Total_Phosphorus_Units', 'Secchi_Depth_Units').count())
unit.toPandas()

,Total_Phosphorus_Units,Secchi_Depth_Units,count
0,mg/L,m,48257


one is in mg per liter while the other is meters

## Problem 2 - Find filter and aggregate.

#### Tasks

Build a query that groups and aggregates to find the yearly average values for both phosphorus and secchi depth.  To do this your will want to

1. Filter based on what you learned in **Problem 1.**
2. Make sure that the `END_DATE` has the correct type and extract the year.  
3. Filter to the correct range of years.
4. Now you should group and aggregate to compute the yearly means.  We want to keep both the `LAKE_NAME` and lake ID.  To do this you should group by both, as well as by year.

In [7]:
# Your code here

from pyspark.sql.functions import to_date, year, regexp_extract, avg

approvals = (mces
        .filter(col('Total_Phosphorus_QUALIFIER') == 'Approved')
        .filter(col('Secchi_Depth_QUALIFIER') == 'Approved')
        .filter(col('Secchi_Depth_RESULT').isNotNull())
        .filter(col('Total_Phosphorus_RESULT').isNotNull())
)
approvals.toPandas()

,PROJECT_ID,DATA_SET_TITLE,LAKE_NAME,CITY,COUNTY,DNR_ID_Site_Number,MAJOR_WATERSHED,WATER_PLANNING_AUTHORITY,LAKE_SITE_NUMBER,START_DATE,...,Secchi_Depth_RESULT_SIGN,Secchi_Depth_RESULT,Secchi_Depth_QUALIFIER,Secchi_Depth_Units,Total_Phosphorus_RESULT_SIGN,Total_Phosphorus_RESULT,Total_Phosphorus_QUALIFIER,Total_Phosphorus_Units,longitude,latitude
0,7108,Citizen Assisted Monitoring Program (CAMP) for...,Acorn Lake,Oakdale,Washington,82010200-01,Lower St. Croix River,Valley Branch WD,1,2006-04-16,...,None,1,Approved,m,None,0.156,Approved,mg/L,-92.97171054,45.01655642
1,7108,Citizen Assisted Monitoring Program (CAMP) for...,Acorn Lake,Oakdale,Washington,82010200-01,Lower St. Croix River,Valley Branch WD,1,2006-05-02,...,None,0.66,Approved,m,None,0.107,Approved,mg/L,-92.97171054,45.01655642
2,7108,Citizen Assisted Monitoring Program (CAMP) for...,Acorn Lake,Oakdale,Washington,82010200-01,Lower St. Croix River,Valley Branch WD,1,2006-05-16,...,None,0.66,Approved,m,None,0.141,Approved,mg/L,-92.97171054,45.01655642
3,7108,Citizen Assisted Monitoring Program (CAMP) for...,Acorn Lake,Oakdale,Washington,82010200-01,Lower St. Croix River,Valley Branch WD,1,2006-05-30,...,None,0.5,Approved,m,None,0.029,Approved,mg/L,-92.97171054,45.01655642
4,7108,Citizen Assisted Monitoring Program (CAMP) for...,Acorn Lake,Oakdale,Washington,82010200-01,Lower St. Croix River,Valley Branch WD,1,2006-06-11,...,None,0.5,Approved,m,None,0.058,Approved,mg/L,-92.97171054,45.01655642
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33643,7105,Lakes Monitoring,Zumbra Lake,Victoria,Carver,10004100-01,Mississippi River - Twin Cities,Minnehaha Creek WD,1,2002-08-19,...,None,1.4,Approved,m,~,0.021,Approved,mg/L,-93.66768906,44.88381717
33644,7105,Lakes Monitoring,Zumbra Lake,Victoria,Carver,10004100-01,Mississippi River - Twin Cities,Minnehaha Creek WD,1,2002-09-05,...,None,2.5,Approved,m,~,0.017,Approved,mg/L,-93.66768906,44.88381717
33645,7105,Lakes Monitoring,Zumbra Lake,Victoria,Carver,10004100-01,Mississippi River - Twin Cities,Minnehaha Creek WD,1,2002-09-16,...,None,2.2,Approved,m,~,0.017,Approved,mg/L,-93.66768906,44.88381717
33646,7105,Lakes Monitoring,Zumbra Lake,Victoria,Carver,10004100-01,Mississippi River - Twin Cities,Minnehaha Creek WD,1,2002-10-01,...,None,2.7,Approved,m,~,0.026,Approved,mg/L,-93.66768906,44.88381717


In [8]:
water_quality_by_year = (approvals
    .withColumn('year', regexp_extract(to_date(col('END_DATE')), r'\d{4}', 0))
    .filter((col('year') > 2003) & (col('year') <= 2014))
    .groupBy(col('year'), col('LAKE_NAME'), col('DNR_ID_Site_Number'))
    .agg(avg(col('Secchi_Depth_RESULT')).alias('AvgSecchi'), avg(col('Total_Phosphorus_RESULT')).alias('AvgPhos'))
)
water_quality_by_year.toPandas()


,year,LAKE_NAME,DNR_ID_Site_Number,AvgSecchi,AvgPhos
0,2013,Downs Lake,82011000-01,0.387143,0.175429
1,2004,East Boot Lake,82003400-01,3.102500,0.038286
2,2011,Bass Lake,82012300-01,2.320256,0.048795
3,2008,Capaul Pond,82036500-01,1.001429,0.085286
4,2007,Forest Lake,82015900-01,1.314286,0.033500
...,...,...,...,...,...
1644,2006,South School Section Lake,82015100-01,1.950800,0.048200
1645,2010,Rest Area Pond,82051400-01,0.622000,0.201600
1646,2007,Shields Lake,82016200-01,1.284571,0.215000
1647,2006,Rose Lake,82011200-01,1.052143,0.077357


In [9]:
water_quality_by_year.toPandas().info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1649 entries, 0 to 1648
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   year                1649 non-null   object 
 1   LAKE_NAME           1649 non-null   object 
 2   DNR_ID_Site_Number  1649 non-null   object 
 3   AvgSecchi           1649 non-null   float64
 4   AvgPhos             1649 non-null   float64
dtypes: float64(2), object(3)
memory usage: 64.5+ KB


## Problem 3 - Find years with complete measurements.

Build a query that leads to a list of lake names and codes that fit the following criteria.

1. Only contains years after 2003.
2. Has a none-missing value for both means.
3. Contains both the lake name and the lake code.

You should save this list of lake IDs in a variable named `lakes_w_complete_info` in a file named `lake.py`.  Restart your kernel and confirm that you can import this data.

In [10]:
from pyspark.sql.functions import when, col,sum
from pyspark.sql.types import IntegerType


In [11]:
verify = (water_quality_by_year
        # .filter((col('DNR_ID_Site_Number').isNotNull()))
        .filter(col('AvgPhos').isNotNull())
        .filter(col('AvgSecchi').isNotNull())
        # .filter(col('LAKE_NAME').isNotNull())
)
verify.toPandas()

,year,LAKE_NAME,DNR_ID_Site_Number,AvgSecchi,AvgPhos
0,2013,Downs Lake,82011000-01,0.387143,0.175429
1,2004,East Boot Lake,82003400-01,3.102500,0.038286
2,2011,Bass Lake,82012300-01,2.320256,0.048795
3,2008,Capaul Pond,82036500-01,1.001429,0.085286
4,2007,Forest Lake,82015900-01,1.314286,0.033500
...,...,...,...,...,...
1644,2006,South School Section Lake,82015100-01,1.950800,0.048200
1645,2010,Rest Area Pond,82051400-01,0.622000,0.201600
1646,2007,Shields Lake,82016200-01,1.284571,0.215000
1647,2006,Rose Lake,82011200-01,1.052143,0.077357


In [12]:
newfunc = {'':1}

In [13]:
listedwater = (water_quality_by_year
            .select('DNR_ID_Site_Number', 'year')
            .withColumn('addingstuff', recode('year', newfunc, default=1))
            .groupBy('DNR_ID_Site_Number')
            .agg(sum(col('addingstuff')).alias('addingstuff'))
            .filter(col('addingstuff')==11)
            .toPandas()['DNR_ID_Site_Number'])
sortedwater = list(sorted(listedwater))
# listedwater.toPandas()
listedwater.count()

/home/wil/.pyenv/versions/anaconda3-2022.05/lib/python3.9/site-packages/pyspark/sql/column.py:419: FutureWarning: A column as 'key' in getItem is deprecated as of Spark 3.0, and will not be supported in the future release. Use `column[key]` or `column.key` syntax instead.
  warnings.warn(


49

In [14]:
with open ('lake.py', 'w') as pywrite:
    pywrite.write(f'lake_complete = {(sortedwater)}')

In [15]:
from lake import lake_complete
lake_complete

['02000500-01',
 '10000200-01',
 '10001100-01',
 '10001900-01',
 '10005200-01',
 '10009500-01',
 '10012100-01',
 '13005300-01',
 '19002100-01',
 '19002200-01',
 '19002300-01',
 '19002400-01',
 '19002500-01',
 '19002601-01',
 '19002700-01',
 '19002900-01',
 '19003100-01',
 '19003300-01',
 '19034800-01',
 '19044600-01',
 '27003501-01',
 '27004201-01',
 '27005300-01',
 '27007000-01',
 '27062700-01',
 '27071100-01',
 '70002600-01',
 '82002000-01',
 '82003400-01',
 '82005400-01',
 '82007700-01',
 '82008700-01',
 '82008900-01',
 '82009002-01',
 '82009200-01',
 '82009400-01',
 '82009700-01',
 '82010100-01',
 '82010300-01',
 '82010400-01',
 '82011301-01',
 '82011602-01',
 '82012200-01',
 '82012300-01',
 '82013700-01',
 '82015300-01',
 '82015900-01',
 '82033400-01',
 '82036800-01']

## Problem 4 - Create and write the final water quality table.

Finally, you should filter the table from **Problem 2.** to the lakes with complete information, then write this table to a `parquet` "file" named `water_quality_by_year.parquet`.

In [16]:
confirmation = (water_quality_by_year
        .where(col('DNR_ID_Site_Number').isin(lake_complete) == True)        
)

In [2]:
# Your code here

# %%timeit -n 1 -r 1

# (confirmation
#  .write
#  .partitionBy('year')
#  .mode('overwrite')
#  .parquet('water_quality_by_year.parquet')



## Deliverables.

Be sure to push your final notebook, as well as the final water quality file to GitHub.